In [ ]:
# AoC Day 23, Puzzle 1, works but takes too long with this brute force approach (program took time to lunch :-) )
from copy import deepcopy

def get_data(filename):
    mp = []
    with open(filename, "r") as file:
        for line in file:
            row = []
            line = line.strip()
            for char in line:
                row.append(char)
            mp.append(row)
    return mp

def print_map(mp):
    for row in mp:
        for char in row:
            print(char, end = "")
        print()
    print()
    
def get_starting_point(mp):
    for col, char in enumerate(mp[0]):
        if char == ".":
            return 0, col
        
def get_end_point(mp):
    for col, char in enumerate(mp[-1]):
        if char == ".":
            return len(mp) - 1, col
        
def goto_next_point(mp, current_point, steps):
    global end_point
    global paths
    #print(current_point, steps)
    row, col = current_point
    # Check, if end_point is reached
    if current_point == end_point:
        paths.append(steps)
        print("Reached end point in ", steps, "steps!")
        #print_map(mp)
        return
    # Check, if row, col out of map
    if not(0 <= row < len(mp)):
        return
    if not(0 <= col < len(mp[0])):
        return
    if mp[row][col] in ["O", "#"]:
        return
    if mp[row][col] in ["<", ">", "v", "^"]:
        # Check if current_point is steep slope
        if mp[row][col] == "^":
            mp[row][col] = "O"
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row - 1, col), steps + 1)
        elif mp[row][col] == "v":
            mp[row][col] = "O"
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row + 1, col), steps + 1)
        elif mp[row][col] == "<":
            mp[row][col] = "O"
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row, col - 1), steps + 1)
        elif mp[row][col] == ">":
            mp[row][col] = "O"
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row, col + 1), steps + 1)
    
    else:
        mp[row][col] = "O"
        #Try North
        if row - 1 >= 0 and mp[row - 1][col] in [".", "<", ">", "v", "^"]:
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row - 1, col), steps + 1)
        #Try South
        if row + 1 < len(mp) and mp[row + 1][col] in [".", "<", ">", "v", "^"]:
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row + 1, col), steps + 1)
        #Try West
        if col - 1 >= 0 and mp[row][col - 1] in [".", "<", ">", "v", "^"]:
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row, col - 1), steps + 1)
        #Try East
        if col + 1 < len(mp[0]) and mp[row][col + 1] in [".", "<", ">", "v", "^"]:
            new_mp = deepcopy(mp)
            goto_next_point(new_mp, (row, col + 1), steps + 1)

# Main Program
paths = []
mp = get_data("Puzzle.txt")
starting_point = get_starting_point(mp)
end_point = get_end_point(mp)
#print_map(mp)
#print(starting_point)
#print(end_point)
goto_next_point(mp, starting_point, 0)
print (max(paths))

In [ ]:
# Redo the program with the approach discussed in reddit: Create a graph where the crosspoints in the labyrinth
# are the nodes and the path between these crossings the edge of a weighted graph.
import networkx as nx
from copy import deepcopy

def get_data(filename):
    mp = []
    with open(filename, "r") as file:
        for line in file:
            row = []
            line = line.strip()
            for char in line:
                # Simplification for Puzzle 2: Replace all <, >, ^, v by .
                if char in ["<", ">", "^", "v"]:
                    row.append(".")
                else:
                    row.append(char)
            mp.append(row)
    return mp

def print_map(mp):
    for row in mp:
        for char in row:
            if char == ".":
                print(" ", end = "")
            else:
                print(char, end = "")
        print()
    print()

def get_starting_point(mp):
    for col, char in enumerate(mp[0]):
        if char == ".":
            return 0, col
        
def get_end_point(mp):
    for col, char in enumerate(mp[-1]):
        if char == ".":
            return len(mp) - 1, col

def find_nodes(mp):
    l_nodes = []
    # The edge of the labyrinth does not contain any crossings!
    for row in range(1, len(mp) - 1):
        for col in range(1, len(mp[0]) - 1):
            if mp[row][col] == ".":
                num = 0
                if mp[row - 1][col] == ".":
                    num += 1
                if mp[row + 1][col] == ".":
                    num += 1
                if mp[row][col - 1] == ".":
                    num += 1
                if mp[row][col + 1] == ".":
                    num += 1
                if num > 2:
                    l_nodes.append((row, col))
                    mp[row][col] = "X"
    return mp, l_nodes

def crawl(mp, node):
    global l_nodes
    row, col = node
    mp[row][col] = "O"
    if node in l_nodes:
        return (1, node)
    if mp[row - 1][col] in [".", "X"]:
        weight, end_node = crawl(mp, (row - 1, col))
        weight += 1
        return (weight, end_node)
    elif mp[row + 1][col] in [".", "X"]:
        weight, end_node = crawl(mp, (row + 1, col))
        weight += 1
        return (weight, end_node)
    elif mp[row][col - 1] in [".", "X"]:
        weight, end_node = crawl(mp, (row, col - 1))
        weight += 1
        return (weight, end_node)
    elif mp[row][col + 1] in [".", "X"]:
        weight, end_node = crawl(mp, (row, col + 1))
        weight += 1
        return (weight, end_node)
    else:
        return -1, -1
    
def sort_node(node1, node2):
    if node1[0] < node2[0]:
        return node1, node2
    elif node1[0] > node2[0]:
        return node2, node1
    else:
        if node1[1] < node2[1]:
            return node1, node2
        elif node1[1] > node2[1]:
            return node2, node1
        else:
            print("Error: Startingnode and endnode are the same")
            return node1, node2
        
def find_edges_from_node(mp, node):
    global l_nodes
    row, col = node
    mp[row][col] = "O"
    l_edges = []
    if (row - 1 > 0) and mp[row - 1][col] == ".":
        new_mp = deepcopy(mp)
        weight, end_node = crawl(new_mp, (row - 1, col))
        if not(end_node == -1):
            #print (weight, end_node)
            #print_map(new_mp)
            min_node, max_node = sort_node(node, end_node)
            l_edges.append((min_node, max_node, weight))
    if (row + 1 < len(mp)) and mp[row + 1][col] == ".":
        new_mp = deepcopy(mp)
        weight, end_node = crawl(new_mp, (row + 1, col))
        if not(end_node == -1):
            #print (weight, end_node)
            min_node, max_node = sort_node(node, end_node)
            l_edges.append((min_node, max_node, weight))
            #print_map(new_mp)
    if (col - 1 > 0) and mp[row][col - 1] == ".":
        new_mp = deepcopy(mp)
        weight, end_node = crawl(new_mp, (row, col - 1))
        if not(end_node == -1):
            #print (weight, end_node)
            min_node, max_node = sort_node(node, end_node)
            l_edges.append((min_node, max_node, weight))
            #print_map(new_mp)
    if (col + 1 < len(mp[0])) and mp[row][col + 1] == ".":
        new_mp = deepcopy(mp)
        weight, end_node = crawl(new_mp, (row, col + 1))
        if not(end_node == -1):
            #print (weight, end_node)
            min_node, max_node = sort_node(node, end_node)
            l_edges.append((min_node, max_node, weight))
            #print_map(new_mp)    
    mp[row][col] = "X"    
    return l_edges

def find_edges(mp):
    global l_nodes
    l_edges = []
    
    for node in l_nodes: 
        l_new_edges = find_edges_from_node(mp, node)
        for edge in l_new_edges:
            if not(edge in l_edges):
                l_edges.append(edge)
    
    return l_edges

def rename_edges(l_edges, starting_point, endpoint):
    name = 1
    d_names = {}
    for edge in l_edges:
        node1, node2, weight = edge
        if not (node1 in d_names):
            d_names[node1] = name
            name += 1
        if not (node2 in d_names):
            d_names[node2] = name
            name += 1
    l_new_edges = []
    for edge in l_edges:
        node1, node2, weight = edge
        new_edge = (d_names[node1], d_names[node2], weight)
        l_new_edges.append(new_edge)
    return l_new_edges, d_names[starting_point], d_names[end_point]

def create_d_edges(l_edges):
    d_edges = {}
    for edge in l_edges:
        node1, node2, weight = edge
        d_edges[(node1, node2)] = weight
    return d_edges

def calc_weight(path):
    global d_edges
    weight = 0
    #print(path)
    for i in range(len(path) - 1):
        node1 = path[i]
        node2 = path[i + 1]
        edge = (node1, node2)
        if edge in d_edges:
            weight += d_edges[edge]
        else:
            edge = (node2, node1)
            weight += d_edges[edge]
    return weight
            
    
# Main Program starts here
mp = get_data("Test.txt")
mp, l_nodes = find_nodes(mp)
starting_point = get_starting_point(mp)
l_nodes.append(starting_point)
end_point = get_end_point(mp)
l_nodes.append(end_point)

l_edges = find_edges(mp)
l_edges, starting_point, end_point = rename_edges(l_edges, starting_point, end_point)
d_edges = create_d_edges(l_edges)

G = nx.Graph()
G.add_weighted_edges_from(l_edges)

l_paths = nx.all_simple_paths(G, starting_point, end_point)

max_weight = 0
for path in l_paths:
    weight = calc_weight(path)
    if max_weight < weight:
        max_weight = weight
print("Longest Path:", max_weight)
    
